# Table of contents

<div class="alert alert-block alert-info" style="margin-top: 20px">
    <ol>
        <li><a href="#1.-Importing-Required-Libraries-and-Data"> Importing Required Libraries and Data</a></li>
        <li><a href="#2.-Data-Pre-processing-and-Preparation"> Data Pre-processing and Preparation</a></li>
        </li>
    </ol>
</div>

# Importing Libraries and Data

In [1]:
!pip install rapidfuzz

In [2]:
!pip install spacy

In [3]:
!pip install gensim

In [4]:
!pip install rank_bm25

In [41]:
!pip install fast-autocomplete

In [5]:
# pip installs
# pip install python-Levenshtein
# pip install "fuzzywuzzy==0.18.0"
# !pip install rapidfuzz
# pip install spacy
# pip install gensim
# !pip install python-Levenshtein
# from rank_bm25 import BM25Okapi
# nltk.download('punkt')
# nltk.download('stopwords') 
# nltk.download('wordnet')
# !pip install rank_bm25
# !python -m spacy download en_core_web_lg 

from fuzzywuzzy import fuzz
from fuzzywuzzy import process

import rapidfuzz
from rapidfuzz import process, utils

import spacy
from tqdm import tqdm

from re import sub
from gensim.utils import simple_preprocess

from nltk.tokenize import word_tokenize
import nltk

from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer

from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer


import spacy
from rank_bm25 import BM25Okapi
from tqdm import tqdm


import pandas as pd


In [6]:
# importing the file

df = pd.read_csv('microstrategy_and_dashboard.csv')
df_metric_desc = pd.read_csv('tableau_metric.csv')

display(df)
display(df_metric_desc)

,id,dashboards,caption,datasource,url,Global_usage,users_id_1,user_id_2
0,139444,Ad Analysis,Beer Style Level Value|Beer Style Levels|Beer ...,IRI 52wk 34.0,https:andanddashboards.armadilloinsight.comand...,NaN,NaN,NaN
1,139443,Ad Recap,Beer Style Level Value|Beer Style Levels|Beer ...,IRI 52wk 34.0,https:andanddashboards.armadilloinsight.comand...,NaN,NaN,NaN
2,139434,Aggregate Sales per Pt vs. CWD,Beer Style Level Value|Beer Style Levels|Beer ...,IRI 52wk 34.0|IRI Agg 34.0,https:andanddashboards.armadilloinsight.comand...,NaN,NaN,NaN
3,139466,Brand Control,Brand Family|Brand Levels|Brand|Main Brand Fam...,IRI 52wk 34.0,https:andanddashboards.armadilloinsight.comand...,NaN,NaN,NaN
4,139465,Brand In and Out Scorecard,Segment|Time as Date,IRI 52wk 34.0,https:andanddashboards.armadilloinsight.comand...,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
71,140693,Depletions Summary,Bottle Size|Brand|Brand Quality|Distributor|Ma...,NaN,NaN,NaN,NaN,NaN
72,140694,Inventoryentory MonthLast Year,Bottle Size|Brand Quality|Distributor|Market M...,NaN,NaN,NaN,NaN,NaN
73,140695,Store Universe,Premise Type|State|Store Name|Filtered Stores,NaN,NaN,NaN,NaN,NaN
74,140696,Surveyed Stores,Month|Premise Type|Response Date|Response Upda...,NaN,NaN,NaN,NaN,NaN


,id,metric_name,descriptors
0,0,Sales Measure Selection,Dollar Sales|Dollar Sales|Dollar Sales Change ...
1,1,Brand Levels,Brand Family|Vendor|Brand Family|Brand|Sku|Seg...
2,3,Beer Style Levels,Beer Style Value|Macro Style Value|Mezzo Style...
3,4,Brands and Skew Value,Brand Family|Brand|Segment
4,5,Beer Style,AMBER ALE|AMBER LAGE|BARLEY WINE|BELGIAN ALES|...
...,...,...,...
303,306,Market Month,1|2|3|4|5|6|7|8|9|10|11|12
304,307,Surveyed Stores,2018|2019|2020|2021
305,308,Numeric Response,Number of Responses
306,309,Units Last Year,2018|2019|2020|2021


In [7]:
# assigning populaity to dashborad

import random
df['popularity'] = [random.randint(1,77) for k in df.dashboards]
df.head(2)

,id,dashboards,caption,datasource,url,Global_usage,users_id_1,user_id_2,popularity
0,139444,Ad Analysis,Beer Style Level Value|Beer Style Levels|Beer ...,IRI 52wk 34.0,https:andanddashboards.armadilloinsight.comand...,NaN,NaN,NaN,32
1,139443,Ad Recap,Beer Style Level Value|Beer Style Levels|Beer ...,IRI 52wk 34.0,https:andanddashboards.armadilloinsight.comand...,NaN,NaN,NaN,64


In [8]:
df = df.apply(lambda x: x.astype(str).str.lower())
df_metric_desc = df_metric_desc.apply(lambda x: x.astype(str).str.lower())

In [9]:
# splitting caption and descriptors columns 

df = df[['dashboards','caption']]
df['metric_names'] = df.caption.str.split('|')
df_metric_desc['descriptor'] = df_metric_desc.descriptors.str.split('|')
display(df)
display(df_metric_desc)

,dashboards,caption,metric_names
0,ad analysis,beer style level value|beer style levels|beer ...,"[beer style level value, beer style levels, be..."
1,ad recap,beer style level value|beer style levels|beer ...,"[beer style level value, beer style levels, be..."
2,aggregate sales per pt vs. cwd,beer style level value|beer style levels|beer ...,"[beer style level value, beer style levels, be..."
3,brand control,brand family|brand levels|brand|main brand fam...,"[brand family, brand levels, brand, main brand..."
4,brand in and out scorecard,segment|time as date,"[segment, time as date]"
...,...,...,...
71,depletions summary,bottle size|brand|brand quality|distributor|ma...,"[bottle size, brand, brand quality, distributo..."
72,inventoryentory monthlast year,bottle size|brand quality|distributor|market m...,"[bottle size, brand quality, distributor, mark..."
73,store universe,premise type|state|store name|filtered stores,"[premise type, state, store name, filtered sto..."
74,surveyed stores,month|premise type|response date|response upda...,"[month, premise type, response date, response ..."


,id,metric_name,descriptors,descriptor
0,0,sales measure selection,dollar sales|dollar sales|dollar sales change ...,"[dollar sales, dollar sales, dollar sales chan..."
1,1,brand levels,brand family|vendor|brand family|brand|sku|seg...,"[brand family, vendor, brand family, brand, sk..."
2,3,beer style levels,beer style value|macro style value|mezzo style...,"[beer style value, macro style value, mezzo st..."
3,4,brands and skew value,brand family|brand|segment,"[brand family, brand, segment]"
4,5,beer style,amber ale|amber lage|barley wine|belgian ales|...,"[amber ale, amber lage, barley wine, belgian a..."
...,...,...,...,...
303,306,market month,1|2|3|4|5|6|7|8|9|10|11|12,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]"
304,307,surveyed stores,2018|2019|2020|2021,"[2018, 2019, 2020, 2021]"
305,308,numeric response,number of responses,[number of responses ]
306,309,units last year,2018|2019|2020|2021,"[2018, 2019, 2020, 2021]"


In [10]:
# converting metric names and descriptors in each row values to multiple rows

df_new = df.set_index(['dashboards'])['metric_names'].apply(pd.Series).stack().reset_index().drop('level_1', axis=1).rename(columns={0:'caption'})
df_new['dashboard_names'] = df.dashboards
df_new_2 = df_metric_desc.set_index(['metric_name'])['descriptor'].apply(pd.Series).stack().reset_index().drop('level_1', axis=1).rename(columns={0:'descriptor'})
df_new

,dashboards,caption,dashboard_names
0,ad analysis,beer style level value,ad analysis
1,ad analysis,beer style levels,ad recap
2,ad analysis,beer style,aggregate sales per pt vs. cwd
3,ad analysis,brand family,brand control
4,ad analysis,brand levels,brand in and out scorecard
...,...,...,...
1709,surveyed stores,premise type,NaN
1710,surveyed stores,survey,NaN
1711,store universe count,premise type,NaN
1712,store universe count,state,NaN


In [11]:
# merging metrics and descriptors

df_final = df_new.merge(df_new_2, how = 'inner', left_on ='caption', right_on = 'metric_name')[['dashboards','caption','descriptor']]
df_final


,dashboards,caption,descriptor
0,ad analysis,beer style level value,macro style value
1,ad analysis,beer style level value,mezzo style value
2,ad analysis,beer style level value,micro style valu
3,ad analysis,beer style level value,beer style value
4,ad analysis,beer style level value,altbier
...,...,...,...
236553,surveyed stores,surveyed stores,2018
236554,surveyed stores,surveyed stores,2019
236555,surveyed stores,surveyed stores,2020
236556,surveyed stores,surveyed stores,2021


In [12]:
df_final.isnull().sum()

dashboards    0
caption       0
descriptor    0
dtype: int64

In [13]:
# checking for metric and descriptors

df_final[(df_final['dashboards'] == '') | (df_final['descriptor'] =='2 silos brewing co') | (df_final['caption'] == '2 silos') ]

,dashboards,caption,descriptor
28434,brand control,select main vendor,2 silos brewing co


## Creating Corpus

In [14]:
# lists of dashboard names, metric names and descriptors 

dashboard_names_list = []
metric_names_list = []
metric_desc_list = []

for names in df.dashboards:
    dashboard_names_list.append(names)

for metric in df_metric_desc.metric_name:
    metric_names_list.append(metric)
    
for dashboard_desc in df_metric_desc.descriptors.str.split('|'):
        for desc in dashboard_desc:
            metric_desc_list.append(desc)

# creating corpus
text = dashboard_names_list + metric_names_list + metric_desc_list

In [15]:
text_new =[]
for words in text:
       text_new.append(words.rstrip().lower())
text_new

['ad analysis',
 'ad recap',
 'aggregate sales per pt vs. cwd',
 'brand control',
 'brand in and out scorecard',
 'brands on ad',
 'brewery comparisons',
 'company',
 'company brands over time',
 'r geography',
 'r geography over time',
 'company',
 'categoryandsegmentanalysis',
 'competitive set',
 'dimensions over time',
 'dollar rank comp',
 'dollar ranker with cwd',
 'dollar vs. ad share',
 'dollars vs. growth',
 'feature ad count',
 'footprint',
 'geographic map',
 'item audit',
 'line geogs over time',
 'market index analysis',
 'market index analysis visual',
 'market share',
 'merch efficiency',
 'merch efficiency c&amp;r',
 'merch execution',
 'merch ranker',
 'merch vs. non-merch',
 'our brands',
 'our brands dollar ranker',
 'our packages',
 'package analysis',
 'price analysis',
 'price bucket analysis',
 'price promotion count',
 'quick scorecard',
 'rankers',
 'sales per pt vs. cwd',
 'style analysis',
 'the humbler',
 'topline',
 'velocity',
 'volume rank comp',
 'deplet

In [16]:
# words

print('shape of DataFrame :' + str(df.shape))
print('Unique dasboard names :' + str(df.dashboards.nunique()))

text = [x.lower() for x in text]
print('Number of dashboard titles : ' + str(len(dashboard_names_list)))
print('Number of metric names : ' + str(len(metric_names_list)))
print('Number of descriptors : ' + str(len(metric_desc_list)))

print('Number of unique dashboard titles : ' + str(len(set(dashboard_names_list))))
print('Number of unique metric names : ' + str(len(set(metric_names_list))))
print('Number of unique descriptors : ' + str(len(set(metric_desc_list))))

shape of DataFrame :(76, 3)
Unique dasboard names :76
Number of dashboard titles : 76
Number of metric names : 308
Number of descriptors : 21592
Number of unique dashboard titles : 76
Number of unique metric names : 271
Number of unique descriptors : 19644


## Pre-Processing Steps :
- stop words
- lower case
- stemming
- lemmatization
- noun filter 



## Search Suggestions flow :
1. Fuzzywuzzy suggestion (includes dashboard title, metric names and descriptors)
- Dashboard names extraction 
- Final suggestions (popularity of dashboard)


## Further techniques :
wordnet

In [17]:
# # creating dictionaries of dashboard names, metric names and descriptors

# dict_metric_desc = dict(zip(df_metric_desc['metric_name'] , df_metric_desc['descriptors']))
# dict_dashboard_metric = dict(zip(df['dashboards'],df['caption']))

# for k,v in dict_metric_desc.items():
#     dict_metric_desc[k] = v.split('|')

# for k,v in dict_dashboard_metric.items():
#     dict_dashboard_metric[k] = v.split('|')
    
# # dict_dashboard_metric['Ad Analysis']
# print(dict_dashboard_metric['Ad Analysis'])
# print(dict_metric_desc['Beer Style Level Value'])




## Using Fuzzywuzzy

In [35]:
def dashboard_name_suggestion(df, Fuzzy_suggestions):
    dashboard_names=[]
    for i in Fuzzy_suggestions:
         dashboard_names.append((df.loc[df.caption==i[0]].dashboards))
    return dashboard_names
 #     return df[df['dashboards'].isin(dashboard_names)].sort_values(by = 'popularity',ascending = False)['dashboards'].to_list()               
   

In [36]:
def fuzzywuzzy_search(corpus,input_query):
    for i in [corpus]:
        return process.extract(input_query,i, limit=15)
        
def dashboard_name_suggestion(df, Fuzzy_suggestions):
    dashboard_names=[]
#     for i in Fuzzy_suggestions:
#             dashboard_names = dashboard_names + ((df.loc[df.dashboards.str.lower()==i[0].lower()].dashboards).to_list())
#     for i in Fuzzy_suggestions:
#             dashboard_names = dashboard_names + ((df.loc[df.caption.str.lower()==i[0].lower()].dashboards).to_list())
    for i in Fuzzy_suggestions:
            dashboard_names = dashboard_names + ((df.loc[df.descriptor.str.lower()==i[0].lower()].dashboards).to_list())
    return dashboard_names


def popular_dashboard_suggestion(df,dashboard_names):
    return df[df['dashboards'].isin(dashboard_names)].sort_values(by = 'popularity',ascending = False)['dashboards'].to_list()               


# dashboard=[]
# dashboard.append(df_new.loc[df_new.caption=='segment'].dashboards.to_list())
# dashboard.append(df_new.loc[df_new.caption=='Brand'].dashboards.to_list())
# dashboard

In [39]:
suggestion = fuzzywuzzy_search(text,'Price per unit dollar sales IPA 12 CT 11 OZ can')
suggestion

[('dollar sales', 90.0, 384),
 ('dollar sales', 90.0, 385),
 ('dollar sales', 90.0, 581),
 ('12 ct 11 oz can', 90.0, 680),
 ('dollar sales', 90.0, 1002),
 ('dollar sales', 90.0, 1003),
 ('price per unit', 90.0, 5331),
 ('dollar sales', 90.0, 5333),
 ('12 ct 11 oz can', 90.0, 21312),
 ('24 ct 11 oz can', 86.89655172413794, 761),
 ('24 ct 11 oz can', 86.89655172413794, 21393),
 ('aggregate sales per pt vs. cwd', 85.5, 2),
 ('dollar rank comp', 85.5, 15),
 ('dollar ranker with cwd', 85.5, 16),
 ('dollar vs. ad share', 85.5, 17)]

In [61]:
suggestion = fuzzywuzzy_search(text,'Price per unit dollar sales IPA 12 CT 11 OZ can')

print(suggestion)
dashboard_suggestions = list(dashboard_name_suggestion(df_final,suggestion))

dasboard_names_2 = list(set(dashboard_suggestions))
dasboard_names_2

[('dollar sales', 90.0, 384), ('dollar sales', 90.0, 385), ('dollar sales', 90.0, 581), ('12 ct 11 oz can', 90.0, 680), ('dollar sales', 90.0, 1002), ('dollar sales', 90.0, 1003), ('price per unit', 90.0, 5331), ('dollar sales', 90.0, 5333), ('12 ct 11 oz can', 90.0, 21312), ('24 ct 11 oz can', 86.89655172413794, 761), ('24 ct 11 oz can', 86.89655172413794, 21393), ('aggregate sales per pt vs. cwd', 85.5, 2), ('dollar rank comp', 85.5, 15), ('dollar ranker with cwd', 85.5, 16), ('dollar vs. ad share', 85.5, 17)]


['ad analysis',
 'rankers',
 'quick scorecard',
 'competitive set',
 'dollar ranker with cwd',
 'our brands',
 'merch vs. non-merch',
 'merch ranker',
 'company',
 'r geography over time',
 'categoryandsegmentanalysis',
 'topline',
 'dollar vs. ad share',
 'the humbler',
 'company brands over time',
 'package analysis',
 'merch efficiency c&amp;r',
 'market index analysis',
 'price promotion count',
 'line geogs over time',
 'brands on ad',
 'market share',
 'market index analysis visual',
 'style analysis',
 'brewery comparisons',
 'price analysis',
 'merch execution',
 'dollars vs. growth',
 'dimensions over time',
 'merch efficiency',
 'company ',
 'aggregate sales per pt vs. cwd',
 'sales per pt vs. cwd',
 'our packages',
 'our brands dollar ranker',
 'velocity',
 'volume rank comp',
 'price bucket analysis',
 'dollar rank comp',
 'r geography',
 'footprint',
 'feature ad count']

In [42]:
from fast_autocomplete import AutoComplete

In [59]:
import csv
from fast_autocomplete.misc import read_csv_gen


def get_words():

    words = {}

    for line in csv_gen:
        dashboard_name = line['dashboard_name']
        metric_name = line['metrics_names']
        descriptor_name = line['descriptiors_names']
        if dashboard_name != metrics_names:
            local_words = [model, '{} {} {}'.format(dashboard_name, metric_name,descriptor_name)]
            while local_words:
                word = local_words.pop()
                if word not in words:
                    words[word] = {}
        if make not in words:
            words[make] = {}
    return words

words

'survey number'

In [51]:
!pip install autocorrect

     |████████████████████████████████| 621 kB 3.0 MB/s eta 0:00:01
  Created wheel for autocorrect: filename=autocorrect-2.3.0-py3-none-any.whl size=621586 sha256=ac975e06dce2739568e191ceecdff24550d8f5e50716cd34b35422172afc7a05
  Stored in directory: /Users/ninadmehta/Library/Caches/pip/wheels/fe/6e/8a/4e8bafec0225cfbdf79a0da722b691e4dc5d20d197423e8b28
Successfully built autocorrect


In [52]:
from autocorrect import Speller
 
check = Speller(lang='en')

'bear'

In [22]:
# checking for metric and descriptors

df_final[(df_final['dashboards'] == '') | (df_final['descriptor'] == 'silos') | (df_final['caption'] == 'select main vendor')].dashboards.unique()

array(['brand control'], dtype=object)

In [23]:
df_new[df_new.caption.str.lower() == 'select main vendor']

,dashboards,caption,dashboard_names
43,brand control,select main vendor,the humbler


In [24]:
# checking dashboard_names for metric = 'segment'
string = 'Geography'.lower()
print(df_new.loc[df_new.caption.str.lower() == string])
print(df_new.loc[df_new.caption.str.lower() == 'geography'].count())

                         dashboards    caption  \
11                      ad analysis  geography   
21                         ad recap  geography   
34   aggregate sales per pt vs. cwd  geography   
44                    brand control  geography   
55                     brands on ad  geography   
67              brewery comparisons  geography   
103                     r geography  geography   
113           r geography over time  geography   
123                         company  geography   
133      categoryandsegmentanalysis  geography   
144                 competitive set  geography   
157            dimensions over time  geography   
172                dollar rank comp  geography   
187          dollar ranker with cwd  geography   
198             dollar vs. ad share  geography   
209              dollars vs. growth  geography   
217                feature ad count  geography   
225                  geographic map  geography   
236            line geogs over time  geography   


In [25]:
#df[df['dashboards'].isin(dasboard_names_2)].sort_values(by = 'popularity', ascending = False)

In [26]:
#popular_suggestions = popular_dashboard_suggestion(df,dasboard_names_2)
#print(len(popular_suggestions))
#popular_suggestions

In [27]:
# dashboard_names=[]
# for i in suggestion:
#         print(i[0])
#         for key, value in dict_dashboard_metric.items():
#                 for v in value:
# #                     print(v)
# #                     print(type(i[0]),type(v))
#                     if i[0] == v:
# #                         print('no')
#                         dashboard_names.append(key)
# dashboard_names

In [28]:
def dashboard_name_suggestion(suggestion, dict1,dict2):
    list_of_keys =[]
    for value in suggestion:
      
        list_of_keys.append([key for key, list_of_values in dict1.items()
                if value[0] in list_of_values])
#         list_of_keys.append([key for key, list_of_values in dict2.items()
#                 if value[0] in list_of_values])
        return list_of_keys
    

## Search query conditions ??

- As the user is more concerned about the dashboard title, the search query should first retrieve the most appropriate dashboard name

- If user searches with descriptors or metric then the corresponding dashboard names should be suggested along with metric

- Dictionary with dashboard names as keys,metric_names,descriptors as values & displaying corresponding keys for value in search suggestion

- output string: dashboard name, metric

# Using TF-IDF

In [29]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

search_terms = 'dollar sales'
documents = text

doc_vectors = TfidfVectorizer().fit_transform([search_terms] + documents)

cosine_similarities = linear_kernel(doc_vectors[0:1], doc_vectors).flatten()
document_scores = [item.item() for item in cosine_similarities[1:]]

tuples = list(zip(text,document_scores))
df_scores = pd.DataFrame(tuples, columns = ['text','document_scores'])
df_scores.sort_values('document_scores', ascending =False).head(10)

,text,document_scores
1003,dollar sales,1.000000
5333,dollar sales,1.000000
384,dollar sales,1.000000
385,dollar sales,1.000000
1002,dollar sales,1.000000
581,dollar sales,1.000000
6361,dollar sales selections,0.730471
386,dollar sales change ya,0.708397
5325,dollar sales per pt,0.683217
389,dollar sales per percent last year,0.645156


In [30]:
doc_vectors.nonzero()

(array([    0,     0,     1, ..., 21975, 21976, 21976], dtype=int32),
 array([15514, 11718,  6546, ...,  1212, 16006, 14546], dtype=int32))

# Using Glove's Model

In [31]:
from re import sub
from gensim.utils import simple_preprocess

query_string = 'area'
documents = text

stopwords = ['the', 'and', 'are', 'a']

# From: https://github.com/RaRe-Technologies/gensim/blob/develop/docs/notebooks/soft_cosine_tutorial.ipynb
def preprocess(doc):
    # Tokenize, clean up input document string
    doc = sub(r'<img[^<>]+(>|$)', " image_token ", doc)
    doc = sub(r'<[^<>]+(>|$)', " ", doc)
    doc = sub(r'\[img_assist[^]]*?\]', " ", doc)
    doc = sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', " url_token ", doc)
    return [token for token in simple_preprocess(doc, min_len=0, max_len=float("inf")) if token not in stopwords]

# Preprocess the documents, including the query string
corpus = [preprocess(document) for document in documents]
query = preprocess(query_string)




# Build the term dictionary, TF-idf model
dictionary = Dictionary(corpus+[query])
tfidf = TfidfModel(dictionary=dictionary)

# Create the term similarity matrix.  
similarity_matrix = SparseTermSimilarityMatrix(similarity_index, dictionary, tfidf)

# Compute Soft Cosine Measure between the query and the documents.
# From: https://github.com/RaRe-Technologies/gensim/blob/develop/docs/notebooks/soft_cosine_tutorial.ipynb
query_tf = tfidf[dictionary.doc2bow(query)]

index = SoftCosineSimilarity(
            tfidf[[dictionary.doc2bow(document) for document in corpus]],
            similarity_matrix)

doc_similarity_scores = index[query_tf]

# Output the sorted similarity scores and documents
sorted_indexes = np.argsort(doc_similarity_scores)[::-1]
for idx in sorted_indexes:
    print(f'{idx} \t {doc_similarity_scores[idx]:0.3f} \t {documents[idx]}')


NameError: name 'Dictionary' is not defined

In [ ]:
# import gensim.downloader as api
# from gensim.corpora import Dictionary
# from gensim.models import TfidfModel
# from gensim.models import WordEmbeddingSimilarityIndex
# from gensim.similarities import SparseTermSimilarityMatrix
# from gensim.similarities import SoftCosineSimilarity

# # Load the model: this is a big file, can take a while to download and open
# glove = api.load("glove-wiki-gigaword-50")    
# similarity_index = WordEmbeddingSimilarityIndex(glove)



In [ ]:
# Build the term dictionary, TF-idf model
dictionary = Dictionary(corpus+[query])
tfidf = TfidfModel(dictionary=dictionary)

# Create the term similarity matrix.  
similarity_matrix = SparseTermSimilarityMatrix(similarity_index, dictionary, tfidf)

# Compute Soft Cosine Measure between the query and the documents.
# From: https://github.com/RaRe-Technologies/gensim/blob/develop/docs/notebooks/soft_cosine_tutorial.ipynb
query_tf = tfidf[dictionary.doc2bow(query)]

index = SoftCosineSimilarity(
            tfidf[[dictionary.doc2bow(document) for document in corpus]],
            similarity_matrix)

doc_similarity_scores = index[query_tf]

# Output the sorted similarity scores and documents
sorted_indexes = np.argsort(doc_similarity_scores)[::-1]
for idx in sorted_indexes:
    print(f'{idx} \t {doc_similarity_scores[idx]:0.3f} \t {documents[idx]}')


# Docsim

## Other Word Embedding techniques:
BERT, BM25,fasttext, FAISS, SEQ2SEQ, WORD2VEC,NMSLIB

In [ ]:
from IPython.display import Image
Image(filename=r'D:\internship Experience\Fintech 2021\Different_techniques.png', width = 500, height =100)